# Исследование надежности заемщиков


## Общая информация о данных

Импортируем библиотеку pandas и сохраним датафрейм в переменную data.

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

data = pd.read_csv(r'C:\Users\bonny\Desktop\мои_проекты_практикум\проект_2_банки\banking.csv')

Выведем первые 5 строчек датафрейма `data` на экран.

In [2]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Выведем основную информацию о датафрейме с помощью метода `info()`.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Выведем количество пропущенных значений для каждого столбца.

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце мы обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных встречаются артефакты (аномалии) — отрицательное количество дней трудового стажа в столбце `days_employed`. Заменим все отрицательные значения положительными.

In [6]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится для исследования.

Выведем перечень уникальных значений столбца `children`.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены.

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный.

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.

In [14]:
data.duplicated().sum()

54

In [15]:
data = data.drop_duplicates()

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.

In [16]:
data['education'] = data['education'].str.lower()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследуйте данные и ответьте на вопросы

**Задание 19. Есть ли зависимость между количеством детей и возвратом кредита в срок?**

In [22]:
debt_by_children = pd.DataFrame() #создали таблицу для сохранения нужных данных

debt_by_children['people_in_category'] = data['children'].value_counts()
#посчитали, у скольких людей из таблицы заданное количество детей
debt_by_children['debtors'] = data['children'][data['debt']==1].value_counts()
#посчитали, у скольких людей в каждой группе есть задолженность по кредиту
debt_by_children['debtors_percent'] = debt_by_children['debtors']/debt_by_children['people_in_category']*100
#вычислили, какой процент людей в каждой группе имеет задолженность по кредиту

display(debt_by_children.sort_values(['debtors_percent'], ascending=False)) 
#вывели таблицу на экран, отсортировав ее по убыванию доли должников в группе


,people_in_category,debtors,debtors_percent
4,41,4.0,9.756098
2,2052,194.0,9.454191
1,4809,444.0,9.232689
3,330,27.0,8.181818
0,14107,1063.0,7.535266
5,9,NaN,NaN


In [23]:
#выясним, почему в ячейках ['debtors'] и ['debtors_percent'] значения NaN. 

print(data.loc[data['children']==5][data['debt']==1].count().sum()) 
#проверим, есть ли вообще должники среди людей с пятью детьми


0


In [24]:
# должников в этой группе нет, тогда заменим значения NaN на 0
debt_by_children = debt_by_children.fillna(0)
display(debt_by_children.sort_values(['debtors_percent'], ascending=False)) 

,people_in_category,debtors,debtors_percent
4,41,4.0,9.756098
2,2052,194.0,9.454191
1,4809,444.0,9.232689
3,330,27.0,8.181818
0,14107,1063.0,7.535266
5,9,0.0,0.000000


**Вывод:** 

   Как видно из таблицы, наибольшая доля должников по кредитам в группе людей с четырьмя (9,76%), двумя детьми (9,45%) и одним ребенком (9,23%). В группу людей с четырьмя детьми входит всего 41 человек, я считаю, это слишком маленькая выборка, чтобы основывать на ней свои выводы. При этом выборка по двум другим группам достаточно большая. Чаще всего не выплачивают кредит вовремя люди с двумя детьми.
   
   Ни у одного человека с пятью детьми, согласно таблице, нет долгов по кредиту, однако эта группа так мала (в нее входит всего 9 человек), что я не стала бы делать на ее основе каких-либо выводов.
   
   Согласно полученным данным, наиболее надежные заемщики - бездетные. В этой группе 7,5% людей имеют задолженность по кредиту. При этом эта категория самая многочисленная, в ней почти в три раза больше человек, чем в следующей по величине категории. Следовательно, можно сказать, что выборка достаточно большая, чтобы считать результаты репрезентативными.
   
   


**Задание 20. Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [25]:
debt_by_family = pd.DataFrame() #построили таблицу по той же логике, что и в задании 19

debt_by_family['people_in_category'] = data['family_status'].value_counts()
debt_by_family['debtors'] = data['family_status'][data['debt']==1].value_counts()
debt_by_family['debtors_percent'] = debt_by_family['debtors']/debt_by_family['people_in_category']*100

display(debt_by_family.sort_values(['debtors_percent'], ascending=False))


,people_in_category,debtors,debtors_percent
Не женат / не замужем,2796,273,9.763948
гражданский брак,4146,385,9.286059
женат / замужем,12266,927,7.557476
в разводе,1189,84,7.064760
вдовец / вдова,951,63,6.624606


**Вывод:** 

Согласно таблице, чаще всего задолженность по кредиту есть у не женатых/не замужних людей (в 9,76% случаев). На втором месте с отставанием в полпроцента - люди, состоящие в гражданском браке.

Наиболее надежными заемщиками по данным таблицы являются вдовцы и вдовы, в этой группе долги есть у 6,62% людей.

В целом таблица показала закономерность: люди, не бывшие в официальном браке, допускают задолженность по кредиту чаще, чем люди, состоящие в браке или бывшие в браке в прошлом.

**Задание 21. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [26]:
debt_by_income = pd.DataFrame() #построили таблицу по той же логике, что и в задании 19 и 20

debt_by_income['people_in_category'] = data['total_income_category'].value_counts()
debt_by_income['debtors'] = data['total_income_category'][data['debt']==1].value_counts()
debt_by_income['debtors_percent'] = debt_by_income['debtors']/debt_by_income['people_in_category']*100

display(debt_by_income.sort_values(['debtors_percent'], ascending=False))

reference_table = pd.DataFrame() #вывели таблицу с расшифровкой категорий доходов для удобства сравнения
columns = ['E', 'D', 'C', 'B', 'A']
incomes = ['0–30000', '30001–50000', '50001–200000', '200001–1000000', '1000001 и выше']
reference_table['category'] = columns
reference_table['income'] = incomes
display(reference_table)

,people_in_category,debtors,debtors_percent
E,22,2,9.090909
C,15938,1353,8.489145
A,25,2,8.000000
B,5014,354,7.060231
D,349,21,6.017192


,category,income
0,E,0–30000
1,D,30001–50000
2,C,50001–200000
3,B,200001–1000000
4,A,1000001 и выше


**Вывод:** 

Категории Е и А слишком малочисленны, чтобы основывать на них свои выводы.

Среди остальных категорий больше всего должников в группе с доходами от 50001 до 200000 рублей ( почти 8,5%). 

Меньше всего должников (6%) в группе с доходами от 30001 до 50000 рублей.

Интересно, что с люди с относительно высоким доходами обращаются за кредитом гораздо чаще, чем с низкими (из 21525 обращений 20952 приходятся на группы В и С - то есть на людей с доходами от 50000 до 1000000).


**Задание 22. Как разные цели кредита влияют на его возврат в срок?**

In [27]:
debt_by_purpose = pd.DataFrame()

debt_by_purpose['people_in_category'] = data['purpose_category'].value_counts()
debt_by_purpose['debtors'] = data['purpose_category'][data['debt']==1].value_counts()
debt_by_purpose['debtors_percent'] = debt_by_purpose['debtors']/debt_by_purpose['people_in_category']*100

display(debt_by_purpose.sort_values(['debtors_percent'], ascending=False))

,people_in_category,debtors,debtors_percent
операции с автомобилем,4281,400,9.343611
получение образования,3989,369,9.250439
проведение свадьбы,2324,183,7.874355
операции с недвижимостью,10754,780,7.253115


**Вывод:** 

Как показывает таблица, доли должников среди тех, кто обращается за кредитом на операции с авто и образование почти одинаковы (9,34% и 9,25% соответственно), что примерно на 1,5% больше, чем среди тех, кто обращается за кредитом на свадьбу и примерно на 2% больше, чем среди тех, кому нужен кредит на операции с недвижимостью.

Следовательно, можно предположить, что вероятность того, что кредит на операции с недвижимостью будет выплачен в срок, наиболее высока, а на операции с автомобилем - наименее высока.

**Задание 23. Приведите возможные причины появления пропусков в исходных данных.**

*Ответ:* 

В исходной таблице обнаружены пропуски в столбцах 'days_employed' и 'total_income' в 2174 строках. 

Возможно, данные не записались при составлении таблицы из-за технической ошибки.

Также возможно, что потенциальные заемщики сознательно не предоставили информацию о своем стаже и ежемесячном доходе. Например, таким образом они хотели повысить свои шансы на получение кредита (если у них маленький стаж и/или низкий доход).

**Задание 24. Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.**

*Ответ:* 

Иногда ячейки или строки с пропусками можно удалить и не учитывать при расчетах. Однако в нашем случае пропуски имеются в 2174 строках - примерно 10% процентах датасета - это слишком много, не хотелось бы терять так много данных, ведь остальные ячейки этих строк содержат информацию.

Поэтому, чтобы не терять эти строки, отсутствующие количественные переменные можно заполнить либо средним арифметическим, либо медианным значением. Медиана предпочтительнее, так как она устойчива к аномальным значениям и выбросам. Так, если в группе есть один человек с крайне высоким или низким доходом, то средний доход по группе будет искажен в большую или меньшую сторону соответственно, и заполнение ячейки таким показателем исказит расчеты. С медианой же такого не произойдет.


## Общий вывод.

Анализ информации о платежеспособности заемщиков выявил следующие результаты.

1. Самая многочисленная и одновременно самая ответственная группа заемщиков по количеству детей - бездетные (если не считать группу из 9 человек с пятью детьми).

2. Люди, не состоящие в официальном браке, чаще допускают просрочку по кредиту, чем женатые. Наименьшая доля должников при группировке по семейному положению - вдовцы.

3. Люди с меньшим доходом реже берут кредит и реже допускают просрочку платежей, чем люди с высоким доходом.

4. Наименьший процент должников - среди людей, обращающихся за кредитом на недвижимость, следовательно, вероятность, что такой кредит будет выплачен вовремя, высока.

При этом отдельно хочу заметить, что доли должников при любых группировках чаще всего различаются примерно на 1-1,5%, то есть нет какой-то группы, которая кардинально выделялась бы в лучшую или худшую сторону (не считая группы с пятью детьми, о чем я уже упоминала). Следовательно, я считаю, что модель кредитного скоринга, построенная на этих результатах, не всегда будет выдавать верное решение.

Если посмотреть на результаты каждого расчета и по ним составить портрет идеального заемщика, то это будет бездетный (или, наоборот, очень многодетный) вдовец с доходом 30000-50000 рублей, который хочет взять ипотеку. Что-то здесь не так.